In [2]:
%pip install transformers torch flash_attn stripedhyena


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [27]:
import torch
import torch.nn.functional as F
from transformers import AutoConfig, AutoModelForCausalLM
from stripedhyena.tokenizer import CharLevelTokenizer

# Define model and device
model_name = "togethercomputer/evo-1-131k-base"
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Load model configuration
model_config = AutoConfig.from_pretrained(model_name, trust_remote_code=True, revision="1.1_fix")
model_config.use_cache = True

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    config=model_config,
    trust_remote_code=True,
    revision="1.1_fix"
).to(device)

# Initialize the tokenizer
tokenizer = CharLevelTokenizer(512)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [28]:
def compute_log_likelihood(sequence):
    """Computes log-likelihood score for a given sequence."""
    # Tokenize sequence and convert to tensor
    input_ids = torch.tensor(
        tokenizer.tokenize(sequence),
        dtype=torch.int
    ).unsqueeze(0).to(device)

    # Ensure input_ids are of type torch.int64
    input_ids = input_ids.long()

    # Forward pass through the model
    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits  # Logits for token predictions

        # Shift logits and input_ids for next-token prediction
        shift_logits = logits[:, :-1, :].contiguous()
        shift_input_ids = input_ids[:, 1:].contiguous()

        # Compute log probabilities
        log_probs = F.log_softmax(shift_logits, dim=-1)

        # Gather log probabilities corresponding to the actual next tokens
        target_log_probs = log_probs.gather(2, shift_input_ids.unsqueeze(-1)).squeeze(-1)

        # Sum log probabilities to get the total log-likelihood
        log_likelihood = target_log_probs.sum().item()

    return log_likelihood

In [29]:
# Example sequence
ref_seq = "ACGTACGTACGTACGTACGT"
alt_seq = "ACGTACGTTCGTACGTACGT"  # Mutation at position 8 (A -> T)

# Compute log-likelihood score
log_likelihood_ref = compute_log_likelihood(ref_seq)
log_likelihood_alt = compute_log_likelihood(alt_seq)

Initializing inference params...
Initializing inference params...


In [30]:
LLR = log_likelihood_alt - log_likelihood_ref
print(LLR)

-4.125
